In [1]:
import tensorflow as tf

In [2]:
import random

In [3]:
tf.set_random_seed(777)  # reproducibility

In [4]:
import numpy as np

In [5]:
import sklearn.model_selection as sk
import pickle

## 불러오고 데이터 랜덤하게 적게 읽어오기

In [6]:
with open('noise_sounds_log.pickle','rb') as file:
    noise_log = pickle.load(file)

with open('snore_sounds_log.pickle','rb') as file:
    snore_log = pickle.load(file)

In [7]:
snore_log.shape

(10231, 1025, 44, 1)

In [8]:
features = np.concatenate((noise_log, snore_log), axis=0)
#개수만큼 만들기
labels = np.array([[1,0] for _ in range(len(snore_log))]+[[0,1] for _ in range(len(noise_log))])

In [9]:
features.shape

(20548, 1025, 44, 1)

In [10]:
tr_features, ts_features, tr_labels, ts_labels = sk.train_test_split(features,labels,test_size=0.20, random_state = 77)

In [11]:
tr_features.shape

(16438, 1025, 44, 1)

In [12]:
ts_features.shape

(4110, 1025, 44, 1)

In [13]:
ts_labels.shape

(4110, 2)

In [14]:
tr_features[0].shape

(1025, 44, 1)

In [15]:
# 데이터 줄이기

In [16]:
# selected_data_range = random.sample(list(range(10777)),5000)

In [17]:
# selected_data_range.sort()

In [18]:
# 줄이기
# tr_features = tr_features[selected_data_range]
# tr_labels = tr_labels_encode[selected_data_range]

In [19]:
# tr_labels

## 학습

In [20]:
learning_rate = 0.002 # 학습율(수치에 따라 기울기 내려가는 정도 조절 가능)

In [21]:
training_epochs = 50

In [22]:
batch_size = 100

In [23]:
# dropout (keep_prob) rate  0.7~0.5 on training, but should be 1 for testing
keep_prob = tf.placeholder(tf.float32)

In [24]:
keep_prob

<tf.Tensor 'Placeholder:0' shape=<unknown> dtype=float32>

In [25]:
# input place holders
X_img = tf.placeholder(tf.float32, [None, 1025, 44, 1])

In [26]:
X_img

<tf.Tensor 'Placeholder_1:0' shape=(?, 1025, 44, 1) dtype=float32>

In [27]:
Y = tf.placeholder(tf.float32, [None, 2])

In [28]:
Y

<tf.Tensor 'Placeholder_2:0' shape=(?, 2) dtype=float32>

In [29]:
W1 = tf.Variable(tf.random_normal([2, 2, 1, 80], stddev=0.01),name='W1')

In [30]:
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 2, 2, 1], padding='SAME')

In [31]:
L1 = tf.nn.relu(L1)

In [32]:
L1

<tf.Tensor 'Relu:0' shape=(?, 513, 22, 80) dtype=float32>

In [33]:
# 여기서는 input tensor순서대로(1개 h,w,ch)
L1 = tf.nn.max_pool(L1, ksize=[1, 3, 2, 1], strides=[1, 1, 1, 1], padding='SAME')

In [34]:
# stride땜에 줄어듦
L1

<tf.Tensor 'MaxPool:0' shape=(?, 513, 22, 80) dtype=float32>

In [35]:
# L2 ImgIn shape=(?, 60, 14, 80)
W2 = tf.Variable(tf.random_normal([2, 3, 80, 120], stddev=0.01),name="W2")

In [36]:
L2 = tf.nn.conv2d(L1, W2, strides=[1, 2, 2, 1], padding='SAME')

In [37]:
L2 = tf.nn.relu(L2)

In [38]:
L2

<tf.Tensor 'Relu_1:0' shape=(?, 257, 11, 120) dtype=float32>

In [39]:
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 1, 1, 1], padding='SAME')

In [40]:
L2

<tf.Tensor 'MaxPool_1:0' shape=(?, 257, 11, 120) dtype=float32>

In [41]:
L2_flat = tf.reshape(L2, [-1, 257*11*120]) # 스트라이드 주변 1행 1열 추가하여 1번 데이터 학습을 여러번 학습하기 위해서

In [42]:
L2_flat

<tf.Tensor 'Reshape:0' shape=(?, 339240) dtype=float32>

In [43]:
# L4 FC 4x4x128 inputs -> 625 outputs
W4 = tf.get_variable("W4", shape=[257*11*120, 1000], initializer=tf.contrib.layers.xavier_initializer())

/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [44]:
b4 = tf.Variable(tf.random_normal([1000]),name="b4")

In [45]:
L4 = tf.nn.relu(tf.matmul(L2_flat, W4) + b4)

In [46]:
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

In [47]:
L4

<tf.Tensor 'dropout/mul:0' shape=(?, 1000) dtype=float32>

In [48]:
# L4 FC 4x4x128 inputs -> 625 outputs
W5 = tf.get_variable("W5", shape=[1000, 80], initializer=tf.contrib.layers.xavier_initializer())

In [49]:
b5 = tf.Variable(tf.random_normal([80]),name="b5")

In [50]:
L5 = tf.nn.relu(tf.matmul(L4, W5) + b5)

In [51]:
L5 = tf.nn.dropout(L5, keep_prob=keep_prob)

In [52]:
L5

<tf.Tensor 'dropout_1/mul:0' shape=(?, 80) dtype=float32>

In [53]:
# L5 Final FC 625 inputs -> 10 outputs
W6 = tf.get_variable("W6", shape=[80, 2], initializer=tf.contrib.layers.xavier_initializer())

In [54]:
b6 = tf.Variable(tf.random_normal([2]),name="b6")

In [55]:
logits = tf.matmul(L5, W6) + b6

In [56]:
logits

<tf.Tensor 'add_2:0' shape=(?, 2) dtype=float32>

In [57]:
# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))

In [58]:
tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)

<tf.Tensor 'Reshape_6:0' shape=(?,) dtype=float32>

In [59]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [60]:
optimizer

<tf.Operation 'Adam' type=NoOp>

In [61]:
# initialize
sess = tf.Session()
saver = tf.train.Saver()

In [62]:
sess.run(tf.global_variables_initializer())

In [63]:
# 계속 random한 값 리턴될수 있도록 하기(으 극혐)
def next_batch(num, data, labels):
    start =0
    randomly_selected_batch_idx = list(range(len(data)))
    random.shuffle(randomly_selected_batch_idx)
    
    for i in range(int(len(data)/num)):
        batch_idx = randomly_selected_batch_idx[start : start + num]
#         print(labels[batch_idx])s
        yield data[batch_idx],labels[batch_idx]
        start = start + num

In [64]:
# train my model
print('Learning started. It takes sometime.')

Learning started. It takes sometime.


In [65]:
# for batch_xs, batch_ys in next_batch(batch_size, tr_features, tr_labels):
#     print(batch_ys.shape)

In [66]:
tr_labels.shape

(16438, 2)

In [67]:
tr_features.shape

(16438, 1025, 44, 1)

In [68]:
import json
hi = {}

In [69]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
tr_features.shape

(16438, 1025, 44, 1)

In [ ]:
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(tr_features.shape[0] / batch_size)

    for batch_xs, batch_ys in next_batch(batch_size, tr_features, tr_labels):
#         print(batch_xs.shape, batch_ys.shape)
        feed_dict = {X_img: batch_xs, Y: batch_ys, keep_prob: 0.5}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
    if epoch%5==0:
        accuracy_num = sess.run(accuracy, feed_dict={X_img:ts_features, Y:ts_labels, keep_prob:1})
        print('--accuracy', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(accuracy_num))
        hi.update({epoch:avg_cost, 'accuracy':float(accuracy_num)})
    else:
        hi.update({epoch:avg_cost})
        
    with open('data.txt', 'w') as outfile:
        json.dump(hi, outfile)
        
    if (epoch+1)%10==0:
        saver.save(sess, "./log_short", global_step=epoch+1)
    
print('Learning Finished!')

Epoch: 0001 cost = 2.939762530
--accuracy 0001 cost = 0.895377100
Epoch: 0002 cost = 0.256043833
Epoch: 0003 cost = 0.250395121
Epoch: 0004 cost = 0.154265839
Epoch: 0005 cost = 0.131444436


## 데이터 저장 및 불러오기

In [ ]:
#http://goodtogreate.tistory.com/entry/Saving-and-Restoring
# saver = tf.train.Saver()
# saver.save(sess, "paper", global_step=5)


In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))

In [ ]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## 정확도

In [ ]:
print('Accuracy:', sess.run(accuracy, feed_dict={
      X_img: ts_features, Y: ts_labels_encode, keep_prob: 1}))

In [ ]:
# Get one and predict
r = random.randint(0, mnist.test.num_examples - 1)

In [ ]:
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))

In [ ]:
print("Prediction: ", sess.run(tf.argmax(logits, 1), feed_dict={X: mnist.test.images[r:r + 1], keep_prob: 1})) 
# 마지막 test 할때는 keep_prob: 1로 무조건 해야 함

In [ ]:
# saver = tf.train.import_meta_graph('paper-5.meta')

In [ ]:
# with tf.Session() as sess:
#   new_saver = tf.train.import_meta_graph('paper-5.meta')
#   new_saver.restore(sess, tf.train.latest_checkpoint('./'))